<a href="https://colab.research.google.com/github/alvarofpinheiro/pifwia_rf/blob/main/PIFWIA_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Florestas aleatórias ou florestas de decisão aleatória são um método de aprendizado para classificação e regressão que constroem uma infinidade de árvores de decisão no treinamento. Para tarefas de classificação, a saída da floresta aleatória é a classe selecionada pela maioria das árvores. Para de regressão, a previsão média ou média das árvores individuais é retornada. As florestas de decisão não sobreajustam o conjunto de treinamento como as árvores de decisão fazem.

Ho, T.K. Random Decision Forests. Proceedings of the 3rd International Conference on Document Analysis and Recognition, Montreal. https://doi.org/10.4236/blr.2019.104046, 1995.

In [1]:
#instalar biblioteca Orange Canvas
!pip install Orange3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#importar biblioteca Orange Canvas
import Orange

In [3]:
#importar dados do disco local
from google.colab import files  
files.upload()

Saving Simulacao1.csv to Simulacao1 (1).csv


{'Simulacao1.csv': b'VC_NDA_VALORCAUSA,VC_NDA_TIPOPESSOA,VC_NDA_TIPODIVIDA,VC_NDA_SITUACAODIVIDA,VC_NDA_TIPOCOBRANCA,ID_NDA\r\ncontinuous,FISICA JURIDICA,AUTO\\ DE\\ APREENS\xc3\x83O AUTO\\ DE\\ INFRA\xc3\x87\xc3\x83O AUTO\\ DE\\ INFRA\xc3\x87\xc3\x83O\\ CPRH AUTO\\ DE\\ INFRA\xc3\x87\xc3\x83O\\ SIMPLES\\ NACIONAL AUTO\\ LAN\xc3\x87AMENTO\\ SEM\\ PENALIDADE D\xc3\x89BITOS\\ DECLARADOS\\ DO\\ SIMPLES\\ NACIONAL NOTIFICA\xc3\x87\xc3\x83O\\ AUTOM\xc3\x81TICA\\ DE\\ D\xc3\x89BITO NOTIFICA\xc3\x87\xc3\x83O\\ DE\\ D\xc3\x89BITO NOTIFICA\xc3\x87\xc3\x83O\\ DE\\ D\xc3\x89BITO\\ DE\\ IPVA NOTIFICA\xc3\x87\xc3\x83O\\ DE\\ LAN\xc3\x87AMENTO\\ ICD SENTENCA\\ JUDICIAL TERMO\\ CONSTITUI\xc3\x87\xc3\x83O\\ CR\xc3\x89DITO\\ N\xc3\x83O\\ TRIBUT\xc3\x81RIO,ATIVO\\ CANCELADO ATIVO\\ INTIMADO ATIVO\\ REGULAR ATIVO\\ SUSPENSO,AJUIZAMENTO PROTESTO,continuous\r\n,,,,class,meta\r\n18538.5,JURIDICA,TERMO CONSTITUI\xc3\x87\xc3\x83O CR\xc3\x89DITO N\xc3\x83O TRIBUT\xc3\x81RIO,ATIVO REGULAR,AJUIZAMENTO,2259.0\r\n

In [4]:
#instanciar objeto de dados com base no caminho gerado com a importação do arquivo
dados = Orange.data.Table("/content/Simulacao1.csv")

In [5]:
#imprimir os primeiros 5 registros
for d in dados[:5]:
  print(d)

[18538.50, JURIDICA, TERMO CONSTITUIÇÃO CRÉDITO NÃO TRIBUTÁRIO, ATIVO REGULAR | AJUIZAMENTO] {2259.0}
[13329.00, JURIDICA, TERMO CONSTITUIÇÃO CRÉDITO NÃO TRIBUTÁRIO, ATIVO REGULAR | AJUIZAMENTO] {2260.0}
[14246.40, JURIDICA, TERMO CONSTITUIÇÃO CRÉDITO NÃO TRIBUTÁRIO, ATIVO REGULAR | AJUIZAMENTO] {2261.0}
[16945.20, JURIDICA, TERMO CONSTITUIÇÃO CRÉDITO NÃO TRIBUTÁRIO, ATIVO REGULAR | AJUIZAMENTO] {2262.0}
[9087.84, JURIDICA, TERMO CONSTITUIÇÃO CRÉDITO NÃO TRIBUTÁRIO, ATIVO REGULAR | AJUIZAMENTO] {2263.0}


In [6]:
#explorar os metadados
qtde_campos = len(dados.domain.attributes)
qtde_cont = sum(1 for a in dados.domain.attributes if a.is_continuous)
qtde_disc = sum(1 for a in dados.domain.attributes if a.is_discrete)
print("%d metadados: %d continuos, %d discretos" % (qtde_campos, qtde_cont, qtde_disc))
print("Nome dos metadados:", ", ".join(dados.domain.attributes[i].name for i in range(qtde_campos)),)

4 metadados: 1 continuos, 3 discretos
Nome dos metadados: VC_NDA_VALORCAUSA, VC_NDA_TIPOPESSOA, VC_NDA_TIPODIVIDA, VC_NDA_SITUACAODIVIDA


In [7]:
#explorar domínios
dados.domain.attributes

(ContinuousVariable(name='VC_NDA_VALORCAUSA', number_of_decimals=2),
 DiscreteVariable(name='VC_NDA_TIPOPESSOA', values=('FISICA', 'JURIDICA')),
 DiscreteVariable(name='VC_NDA_TIPODIVIDA',
                  values=('AUTO DE APREENSÃO',
                   'AUTO DE INFRAÇÃO',
                   'AUTO DE INFRAÇÃO CPRH',
                   'AUTO DE INFRAÇÃO SIMPLES NACIONAL',
                   'AUTO LANÇAMENTO SEM PENALIDADE',
                   'DÉBITOS DECLARADOS DO SIMPLES NACIONAL',
                   'NOTIFICAÇÃO AUTOMÁTICA DE DÉBITO',
                   'NOTIFICAÇÃO DE DÉBITO',
                   'NOTIFICAÇÃO DE DÉBITO DE IPVA',
                   'NOTIFICAÇÃO DE LANÇAMENTO ICD',
                   'SENTENCA JUDICIAL',
                   'TERMO CONSTITUIÇÃO CRÉDITO NÃO TRIBUTÁRIO')),
 DiscreteVariable(name='VC_NDA_SITUACAODIVIDA',
                  values=('ATIVO CANCELADO',
                   'ATIVO INTIMADO',
                   'ATIVO REGULAR',
                   'ATIVO SUSPENSO')

In [8]:
#explorar classe
dados.domain.class_var

DiscreteVariable(name='VC_NDA_TIPOCOBRANCA',
                 values=('AJUIZAMENTO', 'PROTESTO'))

In [10]:
#criar amostra
qtde100 = len(dados)
qtde70 = len(dados) * 70 / 100
qtde30 = len(dados) * 30 / 100
print("Qtde 100%:", qtde100)
print("Qtde  70%:", qtde70)
print("Qtde  30%:", qtde30)
amostra = Orange.data.Table(dados.domain, [d for d in dados if d.row_index < qtde70])
print("Registros:", len(dados))
print("Registros:", len(amostra))

Qtde 100%: 3990
Qtde  70%: 2793.0
Qtde  30%: 1197.0


<ipython-input-10-da7d13f9132f>:8: OrangeDeprecationWarning: Direct calls to Table's constructor are deprecated and will be removed. Replace this call with Table.from_list
  amostra = Orange.data.Table(dados.domain, [d for d in dados if d.row_index < qtde70])


Registros: 3990
Registros: 2793


In [11]:
#Técnica Random Forest (RF)
rf = Orange.classification.RandomForestLearner(
    n_estimators=10, 
    criterion='gini', 
    max_depth=None, 
    min_samples_split=2, 
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features='auto', 
    max_leaf_nodes=None, 
    bootstrap=True, 
    oob_score=False, 
    n_jobs=1, 
    random_state=None, 
    verbose=0, 
    class_weight=None, 
    preprocessors=None)

In [12]:
#Ténica k-Nearest Neighbors (KNN)
knn = Orange.classification.KNNLearner(
    n_neighbors=5, 
    metric='euclidean', 
    weights='distance', 
    algorithm='auto', 
    metric_params=None, 
    preprocessors=None)

In [13]:
#Técnica Support Vector Machine (SVM)
svm = Orange.classification.SVMLearner(
    C=1.0, 
    kernel='rbf', 
    degree=3, 
    gamma='auto', 
    coef0=0.0, 
    shrinking=True, 
    probability=False, 
    tol=0.001, 
    cache_size=200, 
    max_iter=-1, 
    preprocessors=None)

In [14]:
#validar o aprendizado para as 3 técnicas
aprendizado = [rf, svm, knn]
avaliacao = Orange.evaluation.CrossValidation(dados, aprendizado, k=5)

<ipython-input-14-06244bb34849>:3: DeprecationWarning: calling Validation's constructor with data and learners is deprecated;
construct an instance and call it
  avaliacao = Orange.evaluation.CrossValidation(dados, aprendizado, k=5)


In [15]:
#imprimir os indicadores para as 3 técnicas
print(" " * 10 + " | ".join("%-4s" % learner.name for learner in aprendizado))
print("Acurácia  %s" % " | ".join("%.2f" % s for s in Orange.evaluation.CA(avaliacao)))
print("Precisão  %s" % " | ".join("%.2f" % s for s in Orange.evaluation.Precision(avaliacao)))
print("Revocação %s" % " | ".join("%.2f" % s for s in Orange.evaluation.Recall(avaliacao)))
print("F1        %s" % " | ".join("%.2f" % s for s in Orange.evaluation.F1(avaliacao)))
print("ROC       %s" % " | ".join("%.2f" % s for s in Orange.evaluation.AUC(avaliacao)))

          random forest | svm  | knn 
Acurácia  0.84 | 0.74 | 0.73
Precisão  0.85 | 0.75 | 0.74
Revocação 0.83 | 0.73 | 0.70
F1        0.84 | 0.74 | 0.72
ROC       0.90 | 0.74 | 0.81
